In [4]:
%load_ext autoreload
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn enhances the aesthetics of matplotlib plots

import pandas as pd
import xarray as xr
import seaborn as sns
import warnings
import logging
# sns.set()

from tempest import casestudy
from tempest import grid
from tempest import storm_tracker
from tempest import joint_distrib
from tempest import handler
from tempest.plots.hist import simple_hist


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# settings_path = 'settings/sam_4km_30min_30d.yaml'
settings_paths = ["settings/arpege_summer_30d.yaml", "settings/fv3_summer_30d.yaml" , "settings/ifs_summer_30d.yaml" , "settings/mpas_summer_30d.yaml", 
                    "settings/nicam_summer_30d.yaml", "settings/obs_summer_30d.yaml", "settings/sam_summer_30d.yaml", #"settings/sam_4km_30min_30d.yaml",
                    "settings/um_summer_30d.yaml"]


hdlrs = [handler.Handler(settings_path) for settings_path in settings_paths]
css = [casestudy.CaseStudy(hdlr, overwrite = False ,verbose = False) for hdlr in hdlrs]
grs = [grid.Grid(cs, fast = True, overwrite= False, verbose_steps = False, verbose = False) for cs in css]
jds = [joint_distrib.JointDistribution(gr, None, var_id_1 = "mean_unweighted_Prec", var_id_2 = "cond_alpha_85_Prec", 
        nbpd = 20,  nd=5, overwrite = False, dist_mask = False) for gr in grs]

lm = grs[0].get_landmask()
ocean = lm.Landmask == 0
ocean = ocean.values[:,:,np.newaxis]

Data loaded from /homedata/mcarenso/tempest/ARPEGE_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/FV3_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/IFS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/MPAS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/NICAM_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/OBS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/SAM_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/UM_lowRes_Tropics/var_id_days_i_t.json
Found grid attributes file , so loading /homedata/mcarenso/tempest/ARPEGE_lowRes_Tropics/grid_attributes.pkl instead of computing
Found grid attributes file , so loading /homedata/mcarenso/tempest/FV3_lowRes_Tropics/grid_attributes.pkl instead of computing
Found grid attributes file , so loading /homedata/mcarenso/tempest

In [21]:
rank_conv_min = 0
alpha = 0.15
var_alpha = "cond_alpha_"+str(np.round(1-alpha, 2))[2:]+"_Prec"

In [22]:
for jd in jds : 
    jd.make_mask()
    mask_rank_conv = np.outer(jd.dist1.ranks >= 0, jd.dist2.ranks >= rank_conv_min)
    mask_yxt = jd.get_mask_yxt_from_mask_jdist(mask_rank_conv)
    V_rank_alpha = 100*np.sum((jd.prec.nS*jd.prec["Sigma_"+var_alpha]*jd.prec[var_alpha]).values[mask_yxt])/np.sum(jd.prec.nS*jd.prec.mean_unweighted_Prec).values
    print(V_rank_alpha, "%")

15.532534656955702 %
15.40416816999331 %
15.630628700633284 %
15.140097121999236 %
16.003273318302117 %
15.332687228239319 %
15.700108551594713 %
15.651922781104117 %
